In [ ]:
# default_exp runner

In [ ]:
#export
import fire
from pathlib import Path
from datetime import datetime
from pathlib import Path
from functools import partial
from fastcore.script import *
from tegrastatsjson.command_grabber import *
from tegrastatsjson.parser import *
from nbdev.imports import *
from nbdev.export import *

# Tegrastats callback
reveices the line, parses the data and adds a datetime timestamp for the sample
note, the timestamp is when the callback is called and not when the actual sample is received from tegrastats. but this timelag might not be an issue.

In [ ]:
#export
def tegrastats_callback(tegra_line, filename='/var/output/tegra.json'):
    timestamp = str(datetime.now())
    d = ParseTegrastats.parse_line(tegra_line)
    d['timestamp'] = timestamp
    filename = filename
    with open(filename, 'a') as f:
        json.dump(d, f)
        f.write('\n')

In [ ]:
#export
@call_parse
def tegrastatsjson(log_file:Param("path to filename to log to", str)=None, 
                   interval:Param("in ms", int)=1000,  
                   add_date_to_logfile:Param('adds timestamp to filename help distinguish between different runs',bool)=True):
    p = Path(log_file)
    if add_date_to_logfile:
        fname = p.stem+ datetime.now().strftime("-%Y-%m-%d %H:%M:%S") + p.suffix
        p = p.parent / fname
        
    tegrastats_c = partial(tegrastats_callback, filename=str(p))
    args = ['sh', '-c', f"tegrastats --interval {interval}"]
    
    task = CommandGrabber(args, tegrastats_c)
    task.wait_until_finished()
    return
    

In [ ]:
from nbdev.export import *
notebook2script()

ModuleNotFoundError: No module named 'nbdev'